# 1. INTRODUCTION

=================================================

## Milestone 3 Great Expectation

- Nama  : Gilang Pramata Wardhana
- Batch : RMT-043

Nalasia great expectation ini betujuan untuk mengecek dan memastikan kualitas dari data yang akan dianalisa lebih lanjut

=================================================

In [1]:
# Melakukan downgrade great expectation ke versi 0.18.19

!pip install -q "great-expectations==0.18.19"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.6/813.6 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 37.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


# 2. Import Libraries

In [3]:
import os
import pandas as pd
import zipfile
from pathlib import Path
import great_expectations as ge
from great_expectations.core.batch import BatchRequest
from great_expectations.core.expectation_configuration import ExpectationConfiguration

# 3. Load Data

In [4]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
# Proses download data dari kaggle
%%time
!kaggle datasets download -d zahidmughal2343/supplement-sales-data

Dataset URL: https://www.kaggle.com/datasets/zahidmughal2343/supplement-sales-data
License(s): apache-2.0
  0% 0.00/65.2k [00:00<?, ?B/s]
100% 65.2k/65.2k [00:00<00:00, 175MB/s]
CPU times: user 39.8 ms, sys: 3.61 ms, total: 43.4 ms
Wall time: 1.51 s


In [6]:
# Ekstrak file zip
with zipfile.ZipFile('supplement-sales-data.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [7]:
# Cari nama file CSV yang diekstrak
csv_files = [f for f in os.listdir('data') if f.endswith('.csv')]
csv_file = os.path.join('data', csv_files[0]) if csv_files else None

if not csv_file:
    raise FileNotFoundError("Tidak menemukan file CSV dalam dataset")

In [9]:
# Baca data ke pandas DataFrame
df = pd.read_csv(csv_file)
df

,Date,Product Name,Category,Units Sold,Price,Revenue,Discount,Units Returned,Location,Platform
0,2020-01-06,Whey Protein,Protein,143,31.98,4573.14,0.03,2,Canada,Walmart
1,2020-01-06,Vitamin C,Vitamin,139,42.51,5908.89,0.04,0,UK,Amazon
2,2020-01-06,Fish Oil,Omega,161,12.91,2078.51,0.25,0,Canada,Amazon
3,2020-01-06,Multivitamin,Vitamin,140,16.07,2249.80,0.08,0,Canada,Walmart
4,2020-01-06,Pre-Workout,Performance,157,35.47,5568.79,0.25,3,Canada,iHerb
...,...,...,...,...,...,...,...,...,...,...
4379,2025-03-31,Melatonin,Sleep Aid,160,47.79,7646.40,0.21,1,USA,iHerb
4380,2025-03-31,Biotin,Vitamin,154,38.12,5870.48,0.22,1,UK,Walmart
4381,2025-03-31,Green Tea Extract,Fat Burner,139,20.40,2835.60,0.12,3,USA,iHerb
4382,2025-03-31,Iron Supplement,Mineral,154,18.31,2819.74,0.23,2,Canada,Amazon


# 4. Preprocessing

## 4.1 Menghubungkan dengan Dataset

In [25]:
# Inisialisasi context Great Expectations
context = ge.get_context()

INFO:great_expectations.data_context.types.base:Created temporary directory '/tmp/tmp92o08wi0' for ephemeral docs site


In [26]:
# 1. Konfigurasi Datasource - Beri nama unik untuk Datasource
datasource_name = "supplement-sales-data"
datasource = context.sources.add_pandas(datasource_name)

In [27]:
# 2. Konfigurasi Data Asset - Beri nama untuk data asset
asset_name = "supplement-sales-2020-2021"
path_to_data = os.path.join("data", "Supplement_Sales_Weekly_Expanded.csv")

# Tambahkan CSV asset ke datasource
asset = datasource.add_csv_asset(
    asset_name,
    filepath_or_buffer=path_to_data
)

In [28]:
# 3. Membangun Batch Request
batch_request = asset.build_batch_request()

## 4.2 Membuat Expactation Suite

In [30]:
# 4. Membuat Expectation Suite
expectation_suite_name = "supplement_sales_validation"
context.add_or_update_expectation_suite(expectation_suite_name=expectation_suite_name)

# 5. Membuat Validator
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=expectation_suite_name
)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,Date,Product Name,Category,Units Sold,Price,Revenue,Discount,Units Returned,Location,Platform
0,2020-01-06,Whey Protein,Protein,143,31.98,4573.14,0.03,2,Canada,Walmart
1,2020-01-06,Vitamin C,Vitamin,139,42.51,5908.89,0.04,0,UK,Amazon
2,2020-01-06,Fish Oil,Omega,161,12.91,2078.51,0.25,0,Canada,Amazon
3,2020-01-06,Multivitamin,Vitamin,140,16.07,2249.80,0.08,0,Canada,Walmart
4,2020-01-06,Pre-Workout,Performance,157,35.47,5568.79,0.25,3,Canada,iHerb


# 5. Start Expectation

## 5.1. Expectation mengenai Kolom Date

In [33]:
# Ganti expectation uniqueness dengan:
validator.expect_compound_columns_to_be_unique(
    column_list=["Date", "Product Name", "Category", "Location", "Platform"],
    meta={
        "notes": "Memastikan tidak ada record duplikat untuk produk yang sama di lokasi dan platform yang sama pada tanggal yang sama"
    }
)

# Tambahkan expectation untuk memvalidasi format tanggal
validator.expect_column_values_to_match_strftime_format(
    column="Date",
    strftime_format="%Y-%m-%d"
)


  warnings.warn(



Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

  warnings.warn(



Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_match_strftime_format",
    "kwargs": {
      "column": "Date",
      "strftime_format": "%Y-%m-%d",
      "batch_id": "supplement-sales-data-supplement-sales-2020-2021"
    },
    "meta": {}
  },
  "result": {
    "element_count": 4384,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## 5.2. Expectation Units Sold

In [42]:
# Units Sold antara kolom ke dua hingga terahir di 4385
validator.expect_column_values_to_be_between(
    column="Units Sold",
    min_value=2,
    max_value=4385)

  warnings.warn(



Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_be_between",
    "kwargs": {
      "column": "Units Sold",
      "min_value": 2,
      "max_value": 4385,
      "batch_id": "supplement-sales-data-supplement-sales-2020-2021"
    },
    "meta": {}
  },
  "result": {
    "element_count": 4384,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## 5.3. Expectation Category

In [43]:
# Category dalam set yang ditentukan
validator.expect_column_values_to_be_in_set(
    column="Category",
    value_set=["Protein", "Vitamin", "Omega", "Performance", "Amino Acid",
               "Mineral", "Herbal", "Sleep Aid", "Fat Burner", "Hydration"]
)

  warnings.warn(



Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_be_in_set",
    "kwargs": {
      "column": "Category",
      "value_set": [
        "Protein",
        "Vitamin",
        "Omega",
        "Performance",
        "Amino Acid",
        "Mineral",
        "Herbal",
        "Sleep Aid",
        "Fat Burner",
        "Hydration"
      ],
      "batch_id": "supplement-sales-data-supplement-sales-2020-2021"
    },
    "meta": {}
  },
  "result": {
    "element_count": 4384,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## 5.4. Expectation tipe data Price adalah float

In [45]:
# Check parameter float di Price
validator.expect_column_values_to_be_of_type(
    column="Price",
    type_="float64"
)

  warnings.warn(



Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_be_of_type",
    "kwargs": {
      "column": "Price",
      "type_": "float64",
      "batch_id": "supplement-sales-data-supplement-sales-2020-2021"
    },
    "meta": {}
  },
  "result": {
    "observed_value": "float64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## 5.5. Expectation Revenue

In [46]:
# Revenue tidak boleh null
validator.expect_column_values_to_not_be_null(column="Revenue")

  warnings.warn(



Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "Revenue",
      "batch_id": "supplement-sales-data-supplement-sales-2020-2021"
    },
    "meta": {}
  },
  "result": {
    "element_count": 4384,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## 5.6. Expectation Location

In [47]:
# Location harus USA, UK, atau Canada
validator.expect_column_values_to_be_in_set(
    column="Location",
    value_set=["USA", "UK", "Canada"]
)

  warnings.warn(



Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_be_in_set",
    "kwargs": {
      "column": "Location",
      "value_set": [
        "USA",
        "UK",
        "Canada"
      ],
      "batch_id": "supplement-sales-data-supplement-sales-2020-2021"
    },
    "meta": {}
  },
  "result": {
    "element_count": 4384,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## 5.7. Expectation Platform

In [48]:
# Platform harus dalam set yang ditentukan
validator.expect_column_values_to_be_in_set(
    column="Platform",
    value_set=["Walmart", "Amazon", "iHerb"]
)

  warnings.warn(



Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_be_in_set",
    "kwargs": {
      "column": "Platform",
      "value_set": [
        "Walmart",
        "Amazon",
        "iHerb"
      ],
      "batch_id": "supplement-sales-data-supplement-sales-2020-2021"
    },
    "meta": {}
  },
  "result": {
    "element_count": 4384,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [49]:
# Menyimpan hasil dari expectation suite
validator.save_expectation_suite(discard_failed_expectations=False)

# 6. Membuat data Docs

In [50]:
# Membuat checkpoint
checkpoint_name = "supplement_sales_checkpoint"
checkpoint_config = {
    "name": checkpoint_name,
    "config_version": 1,
    "class_name": "SimpleCheckpoint",
    "expectation_suite_name": expectation_suite_name,
    "validations": [
        {
            "batch_request": batch_request,
            "expectation_suite_name": expectation_suite_name
        }
    ]
}

context.add_or_update_checkpoint(**checkpoint_config)

{
  "action_list": [
    {
      "name": "store_validation_result",
      "action": {
        "class_name": "StoreValidationResultAction"
      }
    },
    {
      "name": "store_evaluation_params",
      "action": {
        "class_name": "StoreEvaluationParametersAction"
      }
    },
    {
      "name": "update_data_docs",
      "action": {
        "class_name": "UpdateDataDocsAction"
      }
    }
  ],
  "batch_request": {},
  "class_name": "SimpleCheckpoint",
  "config_version": 1.0,
  "evaluation_parameters": {},
  "expectation_suite_name": "supplement_sales_validation",
  "module_name": "great_expectations.checkpoint",
  "name": "supplement_sales_checkpoint",
  "profilers": [],
  "runtime_configuration": {},
  "validations": [
    {
      "expectation_suite_name": "supplement_sales_validation",
      "batch_request": {
        "datasource_name": "supplement-sales-data",
        "data_asset_name": "supplement-sales-2020-2021",
        "options": {}
      }
    }
  ]
}

In [51]:
# Menjalankan validasi
checkpoint_result = context.run_checkpoint(checkpoint_name=checkpoint_name)

# Menampilkan hasilnya
print(checkpoint_result.get_statistics())

Calculating Metrics:   0%|          | 0/54 [00:00<?, ?it/s]

{'data_asset_count': 1, 'validation_result_count': 1, 'successful_validation_count': 0, 'unsuccessful_validation_count': 1, 'successful_validation_percent': 0.0, 'validation_statistics': {ValidationResultIdentifier::supplement_sales_validation/__none__/20250705T071756.314653Z/supplement-sales-data-supplement-sales-2020-2021: {'evaluated_expectations': 19, 'successful_expectations': 18, 'unsuccessful_expectations': 1, 'success_percent': 94.73684210526315}}}


In [52]:
# Open Data Docs
context.open_data_docs()